In [18]:
import json
from ipywidgets import widgets, Button, VBox, HBox, Textarea, Checkbox, Label
from IPython.display import display, clear_output

# Load JSON data
with open("annotation.json", "r") as f:
    data = json.load(f)

# Check if data is list or dict
if isinstance(data, list):
    entries = data
elif isinstance(data, dict):
    entries = list(data.values())

# Annotated data storage
annotated_data = [{} for _ in range(len(entries))]  # Placeholder for annotations
entry_index = 0  # Global tracker for current entry

# Initialize widgets
key_label = Label("Current Entry:")
entry_editor = Textarea(value="", layout=widgets.Layout(width="80%", height="150px"))  # Smaller text area
bool_selector = Checkbox(value=False, description="is_correct")  # Default to False
save_button = Button(description="Save Changes and Next", button_style="success")
skip_button = Button(description="Skip Entry", button_style="warning")
prev_button = Button(description="Go Back", button_style="primary")
status_label = Label("")

# Function to load an entry
def load_entry():
    global entry_index
    if 0 <= entry_index < len(entries):
        current_entry = entries[entry_index]
        # Load the current entry into the editor
        entry_editor.value = json.dumps(current_entry, indent=4)
        # Load existing annotation if available
        if 'is_correct' in annotated_data[entry_index]:
            bool_selector.value = annotated_data[entry_index]['is_correct']
        else:
            bool_selector.value = False  # Default boolean value
        status_label.value = f"Entry {entry_index + 1} of {len(entries)}"
    else:
        # If all entries are processed, save the data
        clear_output()
        with open("annotated_file.json", "w") as f:
            json.dump(annotated_data, f, indent=4)
        print("Annotation complete! Saved to annotated_file.json")

# Event handler for "Save Changes and Next"
def save_changes_and_next(b):
    global entry_index
    try:
        updated_entry = json.loads(entry_editor.value)  # Parse updated JSON
        updated_entry['is_correct'] = bool_selector.value  # Add boolean key
        annotated_data[entry_index] = updated_entry  # Save changes
        entry_index += 1  # Move to the next entry
        load_entry()
    except Exception as e:
        status_label.value = f"Error saving entry: {e}"

# Event handler for "Skip Entry"
def skip_entry(b):
    global entry_index
    # Ensure that 'is_correct' is explicitly set before skipping
    if bool_selector.value is None:
        status_label.value = "Error: You must set a value for 'is_correct' to skip."
        return
    # Save the current entry without modifications, but include 'is_correct'
    current_entry = entries[entry_index]
    current_entry['is_correct'] = bool_selector.value
    annotated_data[entry_index] = current_entry
    entry_index += 1
    load_entry()

# Event handler for "Go Back"
def go_back(b):
    global entry_index
    if entry_index > 0:
        entry_index -= 1
        load_entry()
    else:
        status_label.value = "Error: You are already at the first entry."

# Bind buttons to handlers
save_button.on_click(save_changes_and_next)
skip_button.on_click(skip_entry)
prev_button.on_click(go_back)

# Display the widgets
load_entry()  # Load the first entry
ui = VBox([
    key_label,
    entry_editor,
    HBox([bool_selector, save_button, skip_button, prev_button]),
    status_label
])
display(ui)

Error in callback <function handle_keypress at 0x7f98fcf8cca0> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f98fc93b190, raw_cell="import json
from ipywidgets import widgets, Button.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://amlext%2B2f737562736372697074696f6e732f35643362356165652d616537652d346463312d396233342d6465616133393933373135332f7265736f7572636547726f7570732f4750552d456173742d55532d322f70726f7669646572732f4d6963726f736f66742e4d616368696e654c6561726e696e6753657276696365732f776f726b7370616365732f4750552d413130302d53482f636f6d70757465732f736861726c616c6b61312d613130302d65617374757332/home/azureuser/localfiles/code/slm-finetuning/fc-finetune-hf/src/annotator/annotator.ipynb#W0sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


TypeError: 'ExecutionInfo' object is not subscriptable